In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
#load csv into gs->(gender stats)
gs = pd.read_csv("gender_statistics.csv")
#drop the column
gs.drop(['Series Code','2020 [YR2020]'], axis = 1, inplace = True)

#find a duplicate rows
duplicateDFrows = gs[gs.duplicated(['Country Name', 'Country Code', 'Series Name'])]
print(duplicateDFrows)

gsdf = gs.rename(columns = {'2018 [YR2018]' : '2018', '2019 [YR2019]' : '2019'}, inplace = False)

#remove null values
gsdf = gsdf.dropna()

# check rows where values are not available, set a flag TRUE/FALSE in another column 
gsdf['flag'] = (gsdf['2019'] == '..') & (gsdf['2019'] == '..')

#drop rows where there are no values
idx = gsdf[gsdf['flag'] == True].index
gsdf.drop(idx, inplace = True)


gsdf.drop('flag', axis = 1, inplace = True)
gsdf.info()

gsdf.to_csv(r'G20_Countries_Economic_Social_stats.csv')

      Country Name Country Code Series Name 2018 [YR2018] 2019 [YR2019]
13105          NaN          NaN         NaN           NaN           NaN
13106          NaN          NaN         NaN           NaN           NaN
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3390 entries, 0 to 12964
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  3390 non-null   object
 1   Country Code  3390 non-null   object
 2   Series Name   3390 non-null   object
 3   2018          3390 non-null   object
 4   2019          3390 non-null   object
dtypes: object(5)
memory usage: 158.9+ KB


# Create dataset to hold economic data of G20 Countries

In [3]:
#create economic indicator list bucket
gdp_list_ = ['GDP (current US$)', 'GDP per capita (Current US$)']

In [4]:
#create dataframe out of population bucket
tempDF = pd.DataFrame(columns = gsdf.columns)
economicDF = []

for item in gdp_list_:
    tempDF = gsdf[gsdf['Series Name'] == item]
    economicDF.append(tempDF)
    #print(populationDF)

economicDF = pd.concat(economicDF)
economicDF.info()
economicDF.count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 152 to 12635
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  42 non-null     object
 1   Country Code  42 non-null     object
 2   Series Name   42 non-null     object
 3   2018          42 non-null     object
 4   2019          42 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


Country Name    42
Country Code    42
Series Name     42
2018            42
2019            42
dtype: int64

In [5]:
#save dataframe to csv
economicDF.to_csv(r'G20_Countries_economic_data.csv')

# Create dataset to hold population data under age 5

In [6]:
#create population list bucket -
list_bucket =['Age population, age 0, female, interpolated',
       'Age population, age 0, male, interpolated',
       'Age population, age 01, female, interpolated',
       'Age population, age 01, male, interpolated',
       'Age population, age 02, female, interpolated',
       'Age population, age 02, male, interpolated',
       'Age population, age 03, female, interpolated',
       'Age population, age 03, male, interpolated',
       'Age population, age 04, female, interpolated',
       'Age population, age 04, male, interpolated',
       'Age population, age 05, female, interpolated',
       'Age population, age 05, male, interpolated']

dict_bucket ={'Age population, age 0, female, interpolated':(0,'Female'),
       'Age population, age 0, male, interpolated':(0,'Male'),
       'Age population, age 01, female, interpolated':(1,'Female'),
       'Age population, age 01, male, interpolated':(1,'Male'),
       'Age population, age 02, female, interpolated':(2,'Female'),
       'Age population, age 02, male, interpolated':(2,'Male'),
       'Age population, age 03, female, interpolated':(3,'Female'),
       'Age population, age 03, male, interpolated':(3,'Male'),
       'Age population, age 04, female, interpolated':(4,'Female'),
       'Age population, age 04, male, interpolated':(4,'Male'),
       'Age population, age 05, female, interpolated':(5,'Female'),
       'Age population, age 05, male, interpolated':(5,'Male')}

In [7]:
#create dataframe out of population bucket
tempDF = pd.DataFrame(columns = gsdf.columns)
populationDF = []

for item in list_bucket:
    tempDF = gsdf[gsdf['Series Name'] == item]
    populationDF.append(tempDF)
    #print(populationDF)

populationDF = pd.concat(populationDF)
populationDF.info()
populationDF.count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 21 to 11888
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  240 non-null    object
 1   Country Code  240 non-null    object
 2   Series Name   240 non-null    object
 3   2018          240 non-null    object
 4   2019          240 non-null    object
dtypes: object(5)
memory usage: 11.2+ KB


Country Name    240
Country Code    240
Series Name     240
2018            240
2019            240
dtype: int64

In [8]:
#function to retun age and gender
def update_age_gender(str):
    for item in dict_bucket.items():      
        if(str == item[0]):
            return(item[1][0],item[1][1])

agecol = []
gendercol = []
for item in map(lambda x: update_age_gender(x), populationDF['Series Name']):
    #print(item[0])
    agecol.append(item[0])
    gendercol.append(item[1])

#add age and gender attribute
populationDF['Age'] = agecol
populationDF['Gender'] = gendercol

populationDF.head()

,Country Name,Country Code,Series Name,2018,2019,Age,Gender
21,Argentina,ARG,"Age population, age 0, female, interpolated",367796,365277,0,Female
645,Australia,AUS,"Age population, age 0, female, interpolated",159460,163020,0,Female
1269,Brazil,BRA,"Age population, age 0, female, interpolated",1438921,1422845,0,Female
1893,Canada,CAN,"Age population, age 0, female, interpolated",188712,193067,0,Female
2517,China,CHN,"Age population, age 0, female, interpolated",7888935,7745542,0,Female


In [10]:
populationDF.to_csv(r'G20_population_age_under5.csv')

# Create dataset for employment information

In [11]:
# bucket two to hold employers & employment attributes    
list_bucket2 = ['Employers, female (% of female employment) (modeled ILO estimate)',
       'Employers, male (% of male employment) (modeled ILO estimate)',
       'Employment in agriculture, female (% of female employment) (modeled ILO estimate)',
       'Employment in agriculture, male (% of male employment) (modeled ILO estimate)',
       'Employment in industry, female (% of female employment) (modeled ILO estimate)',
       'Employment in industry, male (% of male employment) (modeled ILO estimate)',
       'Employment in services, female (% of female employment) (modeled ILO estimate)',
       'Employment in services, male (% of male employment) (modeled ILO estimate)']

dict_bucket2 = {'Employers, female (% of female employment) (modeled ILO estimate)':'Female',
       'Employers, male (% of male employment) (modeled ILO estimate)':'Male',
       'Employment in agriculture, female (% of female employment) (modeled ILO estimate)':'Female',
       'Employment in agriculture, male (% of male employment) (modeled ILO estimate)':'Male',
       'Employment in industry, female (% of female employment) (modeled ILO estimate)':'Female',
       'Employment in industry, male (% of male employment) (modeled ILO estimate)':'Male',
       'Employment in services, female (% of female employment) (modeled ILO estimate)':'Female',
       'Employment in services, male (% of male employment) (modeled ILO estimate)':'Male'}

In [12]:
#create dataframe out of employment bucket
tempDF = pd.DataFrame(columns = gsdf.columns)
employmentnDF = []

for item in list_bucket2:
    tempDF = gsdf[gsdf['Series Name'] == item]
    employmentnDF.append(tempDF)
    #print(populationDF)

employmentnDF = pd.concat(employmentnDF)
employmentnDF.info()
employmentnDF.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 95 to 12582
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Name  168 non-null    object
 1   Country Code  168 non-null    object
 2   Series Name   168 non-null    object
 3   2018          168 non-null    object
 4   2019          168 non-null    object
dtypes: object(5)
memory usage: 7.9+ KB


(168, 5)

In [13]:
employmentnDF.head()

,Country Name,Country Code,Series Name,2018,2019
95,Argentina,ARG,"Employers, female (% of female employment) (mo...",2.6159999370575,2.60899996757507
719,Australia,AUS,"Employers, female (% of female employment) (mo...",3.83599996566772,3.78999996185303
1343,Brazil,BRA,"Employers, female (% of female employment) (mo...",3.3050000667572,3.2720000743866
1967,Canada,CAN,"Employers, female (% of female employment) (mo...",2.70499992370605,2.67600011825562
2591,China,CHN,"Employers, female (% of female employment) (mo...",0.72299998998642,0.745000004768372


In [14]:
#function to retun age and gender
def employment(str):
    #print(str)
    for item in dict_bucket2.items():
        #print(item)
        if(str == item[0]):
            return(item[1])

gendercol = []
for item in map(lambda x: employment(x), employmentnDF['Series Name']):
    #print(item[0])
    gendercol.append(item)

#add age and gender attribute
employmentnDF['Gender'] = gendercol

employmentnDF['2018'] = employmentnDF['2018'].astype(float)
employmentnDF['2019'] = employmentnDF['2019'].astype(float)

employmentnDF.head(50)

,Country Name,Country Code,Series Name,2018,2019,Gender
95,Argentina,ARG,"Employers, female (% of female employment) (mo...",2.616000,2.609000,Female
719,Australia,AUS,"Employers, female (% of female employment) (mo...",3.836000,3.790000,Female
1343,Brazil,BRA,"Employers, female (% of female employment) (mo...",3.305000,3.272000,Female
1967,Canada,CAN,"Employers, female (% of female employment) (mo...",2.705000,2.676000,Female
2591,China,CHN,"Employers, female (% of female employment) (mo...",0.723000,0.745000,Female
3215,France,FRA,"Employers, female (% of female employment) (mo...",2.164000,2.157000,Female
3839,Germany,DEU,"Employers, female (% of female employment) (mo...",2.295000,2.278000,Female
4463,India,IND,"Employers, female (% of female employment) (mo...",0.518000,0.524000,Female
5087,Indonesia,IDN,"Employers, female (% of female employment) (mo...",1.904000,1.929000,Female
5711,Italy,ITA,"Employers, female (% of female employment) (mo...",3.531000,3.524000,Female


In [16]:
employmentnDF.to_csv(r'G20Countries_employment_stats.csv')